In [ ]:

import tensorflow as tf, sys
print("Python:", sys.version.split()[0])
print("TensorFlow:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("GPU available:", len(gpus) > 0, "| GPU devices:", gpus)


In [ ]:

!pip install -q kaggle

from google.colab import files
import os, shutil, glob

print("Upload your Kaggle token file (e.g. 'kaggle (4).json').")
uploaded = files.upload()
kaggle_file = None
for fname in uploaded.keys():
    if 'kaggle' in fname.lower() and fname.lower().endswith('.json'):
        kaggle_file = fname
        break

if kaggle_file is None:
    raise RuntimeError("No kaggle json uploaded. Please upload the file 'kaggle (4).json' (or any kaggle .json).")

kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)
dest = os.path.join(kaggle_dir, "kaggle.json")

if os.path.exists(dest):
    os.remove(dest)
shutil.move(kaggle_file, dest)
os.chmod(dest, 0o600)
print("Kaggle token installed to ~/.kaggle/kaggle.json")


In [ ]:

dataset_slug = "gunavenkatdoddi/eye-diseases-classification"


if os.path.exists("eye-diseases-classification.zip"):
    os.remove("eye-diseases-classification.zip")

print("Downloading dataset:", dataset_slug, " — this may take a moment.")

!kaggle datasets download -d {dataset_slug} -q || echo "kaggle download failed — check token/permission."


zip_files = glob.glob("/content/*.zip")
if not zip_files:
    raise RuntimeError("No zip file found in /content. Check Kaggle download step output above.")
zip_path = zip_files[0]
print("Found zip:", zip_path)


import zipfile
extract_root = "/content/dataset_extracted"
if os.path.exists(extract_root):
    shutil.rmtree(extract_root)
os.makedirs(extract_root, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_root)
print("Extracted dataset to:", extract_root)


In [ ]:

import os

def contains_image_files(path):
    try:
        for f in os.listdir(path):
            if f.lower().endswith(('.jpg','.jpeg','.png')):
                return True
    except Exception:
        return False
    return False

def find_dataset_dir(root):

    children = [os.path.join(root, d) for d in os.listdir(root) if os.path.isdir(os.path.join(root,d))]

    img_count = sum(1 for c in children if contains_image_files(c))
    if img_count >= 2:
        return root

    for dirpath, dirnames, filenames in os.walk(root):
        child_dirs = [os.path.join(dirpath, d) for d in os.listdir(dirpath) if os.path.isdir(os.path.join(dirpath,d))]
        cnt = 0
        for s in child_dirs:
            if contains_image_files(s):
                cnt += 1
        if cnt >= 2:
            return dirpath
    return None

root = "/content/dataset_extracted"
dataset_dir = find_dataset_dir(root)
if dataset_dir is None:

    if os.path.exists("/content/dataset"):
        dataset_dir = "/content/dataset"
    else:
        print("Could not auto-find dataset folder. Listing extracted content for debugging:")
        !ls -R /content/dataset_extracted
        raise RuntimeError("Dataset folder not found automatically. Inspect the listing above.")
else:
    print("Detected dataset directory:", dataset_dir)

final_dataset = "/content/dataset"

if os.path.exists(final_dataset):
    print("Cleaning existing /content/dataset ...")
    shutil.rmtree(final_dataset)
os.makedirs(final_dataset, exist_ok=True)


moved = 0
for entry in os.listdir(dataset_dir):
    s = os.path.join(dataset_dir, entry)
    if os.path.isdir(s):

        shutil.move(s, final_dataset)
        moved += 1

if moved == 0:
    raise RuntimeError(f"No class folders found under {dataset_dir} to move into {final_dataset}.")
print(f"Moved {moved} class folders into {final_dataset}")

print("Final /content/dataset contents:")
!ls -1 /content/dataset


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATASET_PATH = "/content/dataset"
print("Using dataset path:", DATASET_PATH)
print("Top-level folders (classes):", sorted(os.listdir(DATASET_PATH)))

IMG_SIZE = (224,224)
BATCH = 32
VAL_SPLIT = 0.15

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=18,
    zoom_range=0.12,
    horizontal_flip=True,
    validation_split=VAL_SPLIT
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

class_indices = train_gen.class_indices
num_classes = len(class_indices)
print("Detected classes (name -> index):", class_indices)
print("Number of classes:", num_classes)


In [ ]:

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input

base = VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE,3))
base.trainable = False  # freeze weights

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
out = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base.input, outputs=out)
model.summary()


In [ ]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', verbose=1)
]

EPOCHS = 8
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.title("Loss")
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend(); plt.title("Accuracy")
plt.show()


val_steps = int(np.ceil(val_gen.samples / val_gen.batch_size))
y_true = val_gen.classes
y_probs = model.predict(val_gen, steps=val_steps, verbose=1)
y_pred = np.argmax(y_probs, axis=1)


cm = confusion_matrix(y_true, y_pred)

labels = [k for k,v in sorted(class_indices.items(), key=lambda item: item[1])]
print("Labels (index order):", labels)
plt.figure(figsize=(8,6))
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
plt.show()


print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels, zero_division=0))


In [ ]:

index_to_disease = {v: k.strip() for k,v in class_indices.items()}
print("Index -> Disease mapping:", index_to_disease)


In [ ]:



doctor_data = [
    {"name": "Dr. Mehta", "specialization": "Ophthalmologist", "experience": 15, "fee": 800, "success_rate": 96},
    {"name": "Dr. Sharma", "specialization": "Eye Surgeon", "experience": 12, "fee": 900, "success_rate": 92},
    {"name": "Dr. Nair", "specialization": "Optometrist", "experience": 10, "fee": 600, "success_rate": 88},
    {"name": "Dr. Khan", "specialization": "Retina Specialist", "experience": 14, "fee": 1000, "success_rate": 95},
    {"name": "Dr. Gupta", "specialization": "Ophthalmologist", "experience": 8, "fee": 700, "success_rate": 85},
]

disease_specialization = {
    "Bulging Eyes": "Ophthalmologist",
    "Cataracts": "Eye Surgeon",
    "Crossed Eyes": "Optometrist",
    "Glaucoma": "Retina Specialist",
    "Uveitis": "Ophthalmologist",
}

def recommend_doctors_for_disease(disease, top_k=3):
    if disease not in disease_specialization:
        print(f"Error: Disease '{disease}' not found in mapping!")
        return None, []

    specialization = disease_specialization[disease]
    doctors = [d for d in doctor_data if d["specialization"] == specialization]
    sorted_docs = sorted(doctors, key=lambda x: (x["success_rate"], x["experience"]), reverse=True)
    return specialization, sorted_docs[:top_k]


In [ ]:


index_to_disease = {
    0: "Bulging Eyes",
    1: "Cataracts",
    2: "Crossed Eyes",
    3: "Glaucoma",
    4: "Uveitis"
}


In [ ]:



from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

print("Upload an eye image to test the pipeline (jpg/png).")
uploaded = files.upload()

if not uploaded:
    print("No image uploaded. You can also test using validation images (val_gen.filepaths).")
else:
    img_name = list(uploaded.keys())[0]
    img_path = img_name
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, 0)
    preds = model.predict(arr)
    idx = int(np.argmax(preds, axis=1)[0])
    disease = index_to_disease.get(idx, "Unknown").strip()
    prob = float(np.max(preds))
    print(f"Predicted disease: {disease}  (confidence: {prob:.3f})")

    spec, top_docs = recommend_doctors_for_disease(disease, top_k=3)
    print("Mapped specialization:", spec)
    print("Top recommended doctors:")
    for d in top_docs:
        print(f" - {d['name']} ({d['specialization']}), Exp: {d['experience']} yrs, Fees: ₹{d['fee']}, Success: {d['success_rate']}%")
